In [ ]:
from flask import Flask, render_template, request, send_file, jsonify, url_for
import pdfplumber
import pandas as pd
import os
from waitress import serve

app = Flask(__name__)

UPLOAD_FOLDER = "uploads"
OUTPUT_FOLDER = "outputs"
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# Definir funcion para convertir pdf en csv
def extract_pdf_to_csv(pdf_path, csv_output):
    "Extrae tablas de un PDF y las guarda en un archivo CSV."
    with pdfplumber.open(pdf_path) as pdf:
        df = pd.DataFrame(columns=pdf.pages[0].extract_table()[0])  # Crear DataFrame con encabezados
        for page in pdf.pages:
            table = page.extract_table()
            if table:
                num_cols = len(df.columns)
                table = [row + [""] * (num_cols - len(row)) if len(row) < num_cols else row for row in table]
                df = pd.concat([df, pd.DataFrame(table, columns=df.columns)], ignore_index=True)

    # Procesamiento del DataFrame
    df.columns = ["Numero", "Titulo de la norma", "Obligaciones del patron", "Obligaciones de los trabajadores",
                  "Obligaciones generales", "Disposiciones Especificas", "Eliminar"]
    df = df.iloc[3:, :-1]  # Eliminar primeras 3 filas y última columna
    df = df[df.isna().sum(axis=1) < 4]  # Filtrar filas con al menos 4 valores nulos
    df = df.map(lambda x: x.replace("\n", ", ") if isinstance(x, str) else x)  # Sustituir saltos de línea por comas

    df.to_csv(csv_output, index=False)  # Guardar en CSV

# Plantilla html
@app.route("/")
def index():
    return render_template("Plantilla-PDF-CSV-Autogestion.html")

# Cargar archivo
@app.route("/upload", methods=["POST"])
def upload_file():
    if "pdf-file" not in request.files:
        return jsonify({"success": False, "error": "No se seleccionó archivo"}), 400

    file = request.files["pdf-file"]
    if file.filename == "":
        return jsonify({"success": False, "error": "Nombre de archivo inválido"}), 400

    pdf_path = os.path.join(UPLOAD_FOLDER, file.filename)
    file.save(pdf_path)
    
    return jsonify({"success": True, "pdf_path": pdf_path})

# Proceso de conversion de pdf a csv
@app.route("/process", methods=["POST"])
def process_pdf():
    data = request.get_json()
    pdf_path = data.get("pdf_path")

    if not pdf_path or not os.path.exists(pdf_path):
        return jsonify({"success": False, "error": "Archivo PDF no encontrado"}), 400

    csv_output = os.path.join(OUTPUT_FOLDER, os.path.basename(pdf_path).replace(".pdf", ".csv"))
    extract_pdf_to_csv(pdf_path, csv_output)

    return jsonify({"success": True, "csv_url": url_for('download_file', filename=os.path.basename(csv_output), _external=True)})

# Descargar archivo
@app.route("/download")
def download_file():
    filename = request.args.get("filename")
    filepath = os.path.join(OUTPUT_FOLDER, filename)

    if os.path.exists(filepath):
        return send_file(filepath, as_attachment=True)
    
    return jsonify({"error": "Archivo no encontrado"}), 404

if __name__ == '__main__':
    # Usar Waitress para servir la aplicación en lugar de app.run
    serve(app, host='0.0.0.0', port=8000, threads=100)